In [18]:
import pandas as pd
import numpy as np
import datetime

In [31]:
ohlc = pd.read_csv('PML_OHLC.csv').iloc[:,1:]
ohlc.Date = pd.to_datetime(ohlc.Date)
ohlc = ohlc.sort_values(by='Date')

In [113]:
NAV = pd.read_csv('PML_NAV.csv')
NAV.Date = pd.to_datetime(NAV.Date)
NAV.Date = [i + datetime.timedelta(hours=9,minutes=30) for i in NAV.Date]
NAV = NAV.sort_values(by='Date')
df = pd.merge_asof(ohlc,NAV,on='Date',direction='nearest').iloc[:,:7]

In [69]:
dividends = pd.read_csv('PML_dividends.csv')
dividends.Date = pd.to_datetime(dividends.Date)
dividends = dividends.sort_values(by='Date')
dividends.Date = [i + datetime.timedelta(hours=9,minutes=30) for i in dividends.Date]
df = pd.merge_asof(df,dividends,on='Date',direction='nearest')

In [43]:
o_date = ohlc.copy()
o_date['Date2'] = o_date.Date
o_date.drop(['Open','High','Low','Close','Volume'],axis=1,inplace=True)
div_dates = dividends[dividends.Date>=datetime.datetime(2006,1,3)].sort_values(by='Date')
dates = pd.merge_asof(div_dates,o_date,on='Date',direction='nearest')
date = dates.Date2.iloc[0]
ind = 0
since = []
counter = 0
flag = False
for i in range(df.shape[0]):
    if flag == True:
        counter += 1
    if df.Date.iloc[i] == date:
        counter = 0
        if ind < 175:
            ind += 1
            date = dates.Date2.iloc[ind]
        else:
            flag = True
    else:
        counter += 1
    since.append(counter)
til = []
rs = [since[i-1] for i in range(len(since)) if since[i]==0]
til = []
for i in rs:
    til += list(range(i+1))[::-1]
til += [np.nan]*(df.shape[0]-len(til))
df['since'] = since
df['until'] = til

In [44]:
df = df.set_index('Date')
close_prices = df.Close.copy()
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df[col] = df[col].diff()
df = df.dropna()
close_prices = close_prices.loc[df.index]
cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'until',
       'NAV', 'since']
df = df[cols]